In [1]:
! pip install wandb

You should consider upgrading via the '/home/ubuntu/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
from collections import OrderedDict
import math
import numpy as np
import torch
import torch.nn.functional as F
import wandb
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from torch import nn


class swish(nn.Module):
    def __init__(self):
        super(swish, self).__init__()

    def forward(self, input):
        return input * torch.sigmoid(input)


class mish(nn.Module):
    def __init__(self):
        super(mish, self).__init__()

    def forward(self, input):
        return input * torch.tanh(F.softplus(input))
################################################ \phi 1
class arctanish(nn.Module):
    def __init__(self):
        super(arctanish, self).__init__()

    def forward(self, input):
        return input * (2/math.pi * torch.arctan(input) + 1)/2


######################################################\phi 2
class softsignish(nn.Module):
    def __init__(self):
        super(softsignish, self).__init__()

    def forward(self, input):
        return input * ((input /(1.+torch.abs(input)))+1.)/2 


############################################################\phi 3
class loglogish(nn.Module):
    def __init__(self):
        super(loglogish, self).__init__()

    def forward(self, input):
        return input * (1-torch.exp(-torch.exp(input)))

##############################################################\phi 4
class arctanexp(nn.Module):
    def __init__(self):
        super(arctanexp, self).__init__()

    def forward(self, input):
        return input * (2/math.pi * torch.arctan(torch.exp(input)) )



def build_model(activation_function):
    return nn.Sequential(
        OrderedDict(
            [
                ("fc1", nn.Linear(2, 64)),
                ("activation1", activation_function),  # use custom activation function
                ("fc2", nn.Linear(64, 32)),
                ("activation2", activation_function),
                ("fc3", nn.Linear(32, 16)),
                ("activation3", activation_function),
                ("fc4", nn.Linear(16, 1)),
                ("activation4", activation_function),
            ]
        )
    )


def convert_to_PIL(img, width, height):
    img_r = img.reshape(height, width)

    pil_img = Image.new("RGB", (height, width), "red")
    pixels = pil_img.load()

    for i in range(0, height):
        for j in range(0, width):
            pixels[j, i] = img[i, j], img[i, j], img[i, j]

    return pil_img


def main():

    wandb.init(project="landscapes")
    model1 = build_model(nn.ReLU())
    model2 = build_model(swish())
    model3 = build_model(mish())


    model4 = build_model(arctanish())
    model5 = build_model(softsignish())
    model6 = build_model(loglogish())
    model7 = build_model(arctanexp())


    x = np.linspace(0.0, 10.0, num=400)
    y = np.linspace(0.0, 10.0, num=400)

    grid = [torch.tensor([xi, yi], dtype=torch.float) for xi in x for yi in y]

    np_img_relu = np.array([model1(point).detach().numpy() for point in grid]).reshape(
        400, 400
    )
    np_img_swish = np.array([model2(point).detach().numpy() for point in grid]).reshape(
        400, 400
    )
    np_img_mish = np.array([model3(point).detach().numpy() for point in grid]).reshape(
        400, 400
    )



    np_img_arctanish = np.array([model4(point).detach().numpy() for point in grid]).reshape(
    400, 400
    )
    np_img_softsignish = np.array([model5(point).detach().numpy() for point in grid]).reshape(
    400, 400
    )
    np_img_loglogish = np.array([model6(point).detach().numpy() for point in grid]).reshape(
    400, 400
    )
    np_img_arctanexp = np.array([model7(point).detach().numpy() for point in grid]).reshape(
    400, 400
    )
 

    scaler = MinMaxScaler(feature_range=(0, 255))
    np_img_relu = scaler.fit_transform(np_img_relu)
    np_img_swish = scaler.fit_transform(np_img_swish)
    np_img_mish = scaler.fit_transform(np_img_mish)
    
    
    
    np_img_arctanish = scaler.fit_transform(np_img_arctanish)
    np_img_softsignish = scaler.fit_transform(np_img_softsignish)
    np_img_loglogish = scaler.fit_transform(np_img_loglogish)
    np_img_arctanexp = scaler.fit_transform(np_img_arctanexp)
   

    image_relu = convert_to_PIL(np_img_relu, 400, 400)
    image_swish = convert_to_PIL(np_img_swish, 400, 400)
    image_mish = convert_to_PIL(np_img_mish, 400, 400)

    img_arctanish = convert_to_PIL(np_img_arctanish, 400, 400)
    img_softsignish = convert_to_PIL(np_img_softsignish, 400, 400)
    img_loglogish = convert_to_PIL(np_img_loglogish, 400, 400)
    img_arctanexp = convert_to_PIL(np_img_arctanexp, 400, 400)

   

#      wandb.log(
#         {
#             "Output landscapes": [
#                 wandb.Image(np_img_relu, caption="ReLU"),
#                 wandb.Image(np_img_swish, caption="Swish"),
#                 wandb.Image(np_img_mish, caption="Mish"),
#                 wandb.Image(np_img_arctanish, caption="Arctanish"),
#                 wandb.Image(np_img_softsignish, caption="Softsignish"),
#                 wandb.Image(np_img_loglogish, caption="Loglogish"),
#                 wandb.Image(np_img_arctanexp, caption="Arctanexp"),
#                 wandb.Image(np_img_exptanh, caption="Exptanh"),
#             ]
#         }
#     )
    
    wandb.log(
        {
            "Output landscapes": [
                wandb.Image(image_relu, caption="ReLU"),
                wandb.Image(image_swish, caption="Swish"),
                wandb.Image(image_mish, caption="Mish"),
                
                wandb.Image(img_arctanish, caption="Arctanish"),
                wandb.Image(img_softsignish, caption="Softsignish"),
                wandb.Image(img_loglogish, caption="Loglogish"),
                wandb.Image(img_arctanexp, caption="Arctanexp"),
                
            ]
        }
    )


    return


if __name__ == "__main__":
    main()

wandb: Currently logged in as: zhdwu_1989 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
